# 🧠 NeuroGenAI | Story 5: SNN Evaluation Metrics
# Analyze spike train outputs to extract meaningful neuro-behavioral patterns

In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

# 📁 Load spike simulation results

In [ ]:
results_path = "outputs/snn_sim_results.npz"
assert os.path.exists(results_path), f"Spike results not found at {results_path}"
data = np.load(results_path)

spike_times = data["spike_times"]  # in ms
spike_indices = data["spike_indices"]  # neuron ids
voltages = data["voltages"]  # shape: [neurons, timesteps]
print(f"✅ Loaded spike data: {len(spike_indices)} total spikes")

# 📊 Count spikes per neuron

In [ ]:
n_neurons = voltages.shape[0]
spike_counts = np.bincount(spike_indices, minlength=n_neurons)

# 🌟 Calculate firing rate histogram
plt.figure(figsize=(10, 5))
plt.hist(spike_counts, bins=30, color="skyblue", edgecolor="black")
plt.title("Firing Rate Distribution (Spikes per Neuron)")
plt.xlabel("Spike Count")
plt.ylabel("# of Neurons")
os.makedirs("outputs", exist_ok=True)
plt.savefig("outputs/snn_firing_rate_hist.png")
print("📊 Saved firing rate histogram to outputs/snn_firing_rate_hist.png")

# 🔢 Entropy: Diversity of firing

In [5]:
p_distribution = spike_counts / spike_counts.sum()
neuro_entropy = float(entropy(p_distribution, base=2))

# ⚖️ Gini coefficient for spiking imbalance

In [6]:
def gini(x):
    x = np.sort(x)
    n = len(x)
    index = np.arange(1, n+1)
    return (2 * np.sum(index * x) / (n * np.sum(x))) - ((n + 1) / n)

neuro_gini = float(gini(spike_counts))

# 💾 Save JSON metrics

In [ ]:
metrics = {
    "total_spikes": int(len(spike_indices)),
    "neurons": int(n_neurons),
    "mean_firing_rate": float(np.mean(spike_counts)),
    "max_firing_rate": int(np.max(spike_counts)),
    "entropy": neuro_entropy,
    "gini": neuro_gini
}

with open("outputs/snn_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

print("✅ Saved SNN evaluation metrics to outputs/snn_metrics.json")